# Online Markplatz Kaufland<br>
## Main Category ---> Lebensmittel & Gesundheit<br>
### Max PageNumber ---> Any PageNumber<br>

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
import json
import time
import re

In [ ]:


main_url = "https://www.kaufland.de/"
print('Main Website Page', main_url)

def request_to_kaufland(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        return BeautifulSoup(response.text, "html.parser")
    except requests.RequestException as e:
        print("Error:", e)
        return None


def access_main_categories(categories_link):
    print ('Total Number of Categories : ', len(categories_link))
    try:
        for i in range(len(categories_link)):   # len(categories_link)
            print ('Main Category : ', categories_link[i])
            print ('Main Category Link : ', categories_link[i])
            categories_soup = request_to_kaufland(categories_link[i])
            a_tag = categories_soup.find_all('a', class_='rd-link rd-link--navi-secondary rd-category-sublinks__sublink rd-category-sublinks__sublink')
            return a_tag
    except IndexError as e:
        print("Error accessing main categories:", e)
        return []

def access_sub_categories(subcategory_links):
    try:
        print ('Total Number of Sub-Categories within the Category :', len(subcategory_links))
        subsub_categories = []  # Collect subsubcategory links
        for j in range(len(subcategory_links)):
            href_value = subcategory_links[j]['href']
            print ('Sub-Category Link : ', href_value)
            SubCate_soup = request_to_kaufland(href_value)
        #    time.sleep(0.5)
            try:
                Sub_tag = SubCate_soup.find_all('a', class_='rd-link rd-link--navi-secondary rd-category-tree__anchor rd-category-tree__anchor--level-1')
                if len(Sub_tag) == 0:
                    first_page_products(SubCate_soup, PageNumber = 1)
                else:
                    subsub_categories.extend(Sub_tag)  # Collect subsubcategory links
            except:
                Sub_tag = SubCate_soup.find_all('a', class_='rd-link rd-link--navi-secondary rd-category-tree__anchor rd-category-tree__anchor--level-1')
                subsub_categories.extend(Sub_tag)  # Collect subsubcategory links

        return subsub_categories  # Return the list of subsubcategory links
    except (IndexError, KeyError) as e:
        print("Error accessing sub categories:", e)
        return []

def access_subsub_categories(Subsub_categories_links):
    try:
        print ('Lenth of the Sub-Sub categories Links : ', len(Subsub_categories_links))
        subsub_categories = []  # Collect subsubcategory links
        if Subsub_categories_links:
            for k in range(len(Subsub_categories_links)):
                href_value = Subsub_categories_links[k]['href']
                print('Sub-Sub Category Link: ', href_value)
                Sub_SubCate_soup = request_to_kaufland(href_value)
        #        time.sleep(0.5)
                try:
                    Subsub_tag = Sub_SubCate_soup.find_all('a', class_='rd-link rd-link--navi-secondary rd-category-tree__anchor rd-category-tree__anchor--level-2')
                    if len(Subsub_tag) == 0:
                        first_page_products(Sub_SubCate_soup, PageNumber = 1)
                    else:
                        subsub_categories.extend(Subsub_tag)  # Collect subsubcategory links
                except:
                    Subsub_tag = Sub_SubCate_soup.find_all('a', class_='rd-link rd-link--navi-secondary rd-category-tree__anchor rd-category-tree__anchor--level-2')
                    subsub_categories.extend(Subsub_tag)  # Collect subsubcategory links
        else:
            print("No subcategories found.")

        return subsub_categories  # Return the list of subsubcategory links
    except (IndexError, KeyError) as e:
        print("Error accessing sub-sub categories:", e)
        return []

def access_subsubsub_categories(SubSubSub_cat):
    try:
        for l in range(len(SubSubSub_cat)):
            href_value = SubSubSub_cat[l]['href']
            print('Sub-Sub-Sub Category Link: ', href_value)
            Sub_SubCate_soup = request_to_kaufland(href_value)
         #   time.sleep(0.5)
            return Sub_SubCate_soup
    except (IndexError, KeyError) as e:
        print("Error accessing sub-sub-sub categories:", e)
        return []

def first_page_products(ProductLists, PageNumber = 1):
    try:
        product_link = ProductLists.find_all('a', class_='product-link')
        print ('Total Number of Products in Page', PageNumber,' is : ', len(product_link))
        for ii in range(len(product_link)):  # len(product_link)
            href_value = product_link[ii]['href']
         #   print('Link of product:', main_url + href_value[1:])
            product_soup = request_to_kaufland(main_url + href_value[1:])
            product_linkXX = main_url + href_value[1:]
            time.sleep(2.5)
            Extract_Product_Info(product_soup, product_linkXX)

       # search_query_tag = ProductLists.find('script', text=lambda text: text and 'window.__SEARCHFRONTEND__' in text)
        search_query_tag = ProductLists.find('script', string=lambda text: text and 'window.__SEARCHFRONTEND__' in text)
        input_string = str(search_query_tag)
        url_pattern = r'url:"(.*?)"'
        urls = re.findall(url_pattern, input_string)
        number_pattern = r'\\u002Fcategory\\u002F(\d+)\\'
        numbers = []
        for url in urls:
            match = re.search(number_pattern, url)
            if match:
                number = match.group(1)
                numbers.append(number)
        
        if PageNumber <= 4:   # Here we can adjust PageNumber upto any limit
            try:
                A = request_to_kaufland(main_url+'category/'+str(numbers[-1])+'/p'+str(PageNumber+1)+'/')
                print ('Access to the Page Number : ', PageNumber+1, ' link is :', main_url+'category/'+str(numbers[-1])+'/p'+str(PageNumber+1)+'/')
                first_page_products(A, PageNumber +1)
            except:
                print ('Error Due to PageNumber/Category....')   
            '''
            try:
                B = request_to_kaufland(main_url+'topic/'+str(numbers[-1])+'/p'+str(PageNumber+1)+'/')
            #    print ('Access to the Page Number : ', PageNumber+1, ' link is :', main_url+'category/'+str(numbers[-1])+'/p'+str(PageNumber+1)+'/')
                first_page_products(B, PageNumber +1)
            except:
                print ('Error Due to PageNumber/Topic....')  
            '''
    except (IndexError, KeyError) as e:
        print("Error accessing first page products:", e)
        return None

def Extract_Product_Info(product_soup, product_linkXX):
    data_to_save = {}  # Dictionary to store extracted data
    product_name = product_soup.find('h1').text.strip()
    data_to_save['Product_Name'] = product_name
    data_to_save['Product_link'] = product_linkXX
    categories = product_soup.find_all(class_='rd-breadcrumb__item')
    count = 1
    for category in categories:
        data_to_save['Category_'+str(count)] = category.text.strip()
        count += 1

    try:
        price = product_soup.find(class_='rd-price-information__price').text.replace('\xa0', ' ').strip()        #rd-buybox-comparison__base-price
        data_to_save['Price'] = price
    except:
        data_to_save['Price'] = None

    description = product_soup.find(class_='rd-description-teaser__description-text').text
    data_to_save['Description'] = description
    images = str(product_soup.find_all(class_='swiper-slide'))
    https_urls = re.findall(r'srcset="(https://.*?)\"', images)
    count = 1
    for url in https_urls:
        data_to_save['Product_Image_'+str(count)] = url
        count += 1
    
    #other_info = product_soup.find('script', text=lambda text: text and 'window.__PDPFRONTEND__' in text)
    other_info = product_soup.find('script', string=lambda text: text and 'window.__PDPFRONTEND__' in text)
    input_string = str(other_info)        

    patterns = {
        "Verkehrsbezeichnung": r'name:"Verkehrsbezeichnung",values:\[{text:"(.*?)"',
        "Allergene": r'name:"Allergene",values:\[{text:"(.*?)"',
        "EAN": r'name:"EAN",values:\[{text:"(.*?)"',
        "Zutatenverzeichnis": r'name:"Zutatenverzeichnis",values:\[{text:"(.*?)"',
        "Nährwertdeklaration": r'name:"Nährwertdeklaration",values:\[{text:"(.*?)"',
    }

    for key, pattern in patterns.items():
        match = re.search(pattern, input_string)
        if match:
            data_to_save[key] = match.group(1)
        else:
            data_to_save[key] = "Not found"
    save_json(data_to_save)
    
def save_json(data):
    try:
        with open('justfinaltry.json', 'r') as json_file:
            existing_data = json.load(json_file)
    except FileNotFoundError:
        existing_data = []

    existing_data.append(data)

    with open('justfinaltry.json', 'w') as json_file:
        json.dump(existing_data, json_file, indent=4)

 #   print("Data appended to Kauland_Lebensmittlel.json")

soup = request_to_kaufland(main_url)
if soup:
    categories_link = ["https://www.kaufland.de/lebensmittel/",
                       "https://www.kaufland.de/koerperpflege-und-gesundheit/"
                      ]
    subcategory_links = access_main_categories(categories_link)
    Subsub_categories_links = access_sub_categories(subcategory_links)
    print ('Problem1111111..............')
    SubSubSub_cat = access_subsub_categories(Subsub_categories_links)
    if SubSubSub_cat:
        print ('Problem222222222..............')
        ProductLists = access_subsubsub_categories(SubSubSub_cat)
        product_soup = first_page_products(ProductLists)

    else:
        print ('Problem 3........................')
else:
    print("Failed to retrieve data from the main page.")


Main Website Page https://www.kaufland.de/
Total Number of Categories :  2
Main Category :  https://www.kaufland.de/lebensmittel/
Main Category Link :  https://www.kaufland.de/lebensmittel/
Total Number of Sub-Categories within the Category : 55
Sub-Category Link :  https://www.kaufland.de/tee/
Total Number of Products in Page 1  is :  35
Access to the Page Number :  2  link is : https://www.kaufland.de/category/2481/p2/
Total Number of Products in Page 2  is :  35
Access to the Page Number :  3  link is : https://www.kaufland.de/category/2481/p3/
Total Number of Products in Page 3  is :  35
Access to the Page Number :  4  link is : https://www.kaufland.de/category/2481/p4/
Total Number of Products in Page 4  is :  35
Error Due to PageNumber/Category....
Sub-Category Link :  https://www.kaufland.de/loser-tee/
Total Number of Products in Page 1  is :  35
Sub-Category Link :  https://www.kaufland.de/teebeutel/
Total Number of Products in Page 1  is :  35
Sub-Category Link :  https://www.